### Serial Tree Creation, Parallel Forest Creation

In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()
import numpy as np

Let's begin with a small sample.

In [2]:
sample=[('1',[1,1,0,1]),
         ('0',[0,0,1,1]),
         ('1',[1,1,0,2]),
         ('0',[1,1,0,1]),
         ('0',[0,0,1,0]),
         ('2',[1,0,2,1]),
         ('2',[2,0,1,1]),
         ('2',[1,2,1,1]),
         ('0',[0,1,1,1]),
         ('1',[1,2,0,2]),
         ('0',[1,1,1,1]),
         ('0',[0,1,1,0]),
         ('2',[2,2,2,1]),
         ('2',[2,1,1,1]),
         ('2',[2,2,1,1])]

col_sample = ['A','B','C','D']

The code below was borrowed from and then adapted for our array of labeledPoints from: http://www-users.cs.umn.edu/~kumar/dmbook/ch4.pdf

In [3]:
"""
This module holds functions that are responsible for creating a new
decision tree and for using the tree for data classificiation.
"""

def majority_value(data):
    """
    Creates a list of all values in the target attribute for each record
    in the data list object, and returns the value that appears in this list
    the most frequently.
    """
    return most_frequent([record[0] for record in data])

def most_frequent(lst):
    """
    Returns the item that appears most frequently in the given list.
    """
    lst = lst[:]
    highest_freq = 0
    most_freq = None

    for val in unique(lst):
        if lst.count(val) > highest_freq:
            most_freq = val
            highest_freq = lst.count(val)
            
    return most_freq

def unique(lst):
    """
    Returns a list made up of the unique values found in lst.  i.e., it
    removes the redundant values in lst.
    """
    lst = lst[:]
    unique_lst = []

    # Cycle through the list and add each value to the unique list only once.
    for item in lst:
        if unique_lst.count(item) <= 0:
            unique_lst.append(item)
            
    # Return the list with all redundant values removed.
    return unique_lst

def get_values(data, attr):
    """
    Creates a list of values in the chosen attribut for each record in data,
    prunes out all of the redundant values, and return the list.  
    """
    return unique([record[1][attr] for record in data])

def choose_attribute(data, attributes, fitness):
    """
    Cycles through all the attributes and returns the attribute with the
    highest information gain (or lowest entropy).
    """
    best_gain = 0.0
    best_attr = None

    for attr in attributes:
        gain = fitness(data, attr)
        if gain >= best_gain:
            best_gain = gain
            best_attr = attr
                
    return best_attr

def get_examples(data, attr, value):
    """
    Returns a list of all the records in <data> with the value of <attr>
    matching the given value.
    """
    rtn_lst = []
    if not data:
        return rtn_lst
    else:
        for record in data:
            if record[1][attr]==value:
                rtn_lst.append(record)
    return rtn_lst

def create_decision_tree(data, attributes, fitness_func, columns):
    """
    Returns a new decision tree based on the examples given.
    """
    vals = [x[0] for x in data]
    default = most_frequent(vals)

    # If the dataset is empty or the attributes list is empty, return the
    # default value. When checking the attributes list for emptiness, we
    # need to subtract 1 to account for the target attribute.
    if not data or (len(attributes) - 1) <= 0:
        return default
    # If all the records in the dataset have the same classification,
    # return that classification.
    elif vals.count(vals[0]) == len(vals):
        return vals[0]
    else:
        # Choose the next best attribute to best classify our data
        best = choose_attribute(data, attributes,
                                fitness_func)

        # Create a new decision tree/node with the best attribute and an empty
        # dictionary object--we'll fill that up next.
        tree = {columns[best]:{}}

        # Create a new decision tree/sub-node for each of the values in the
        # best attribute field
        for val in get_values(data, best):
            # Create a subtree for the current value under the "best" field
            subtree = create_decision_tree(
                get_examples(data, best, val),
                [attr for attr in attributes if attr != best],
                fitness_func, columns)

            # Add the new subtree to the empty dictionary object in our new
            # tree/node we just created.
            tree[columns[best]][val] = subtree

    return tree


In [4]:
-
            


Support function to take care of continuous variables.

In [5]:
##Loading sample data
sample2=[(1,[1,1,30,27,1]),
         (0,[0,0,5,8,1]),
         (1,[1,1,27,16,2]),
         (0,[1,1,2,4,1]),
         (0,[0,0,23,8,0]),
         (2,[1,0,20,18,1]),
         (2,[2,0,15,11,1]),
         (2,[1,2,11,25,1]),
         (0,[0,1,11,23,1]),
         (1,[1,2,8,21,2]),
         (0,[1,1,9,9,1]),
         (0,[0,1,18,2,0]),
         (2,[2,2,7,18,1]),
         (2,[2,1,13,6,1]),
         (2,[2,2,19,8,1])]

In [6]:
from bisect import bisect
def discrete_val(data,ranges):
    k = bisect(ranges,float(data))
    if k == 0:
        return str(ranges[k])+"<"
    elif k == len(ranges):
        return str(ranges[k-1])+">"
    else:
        return str(ranges[k-1])+ "-" + str(ranges[k])  

In [7]:
#Code written according to your input - Not optimized
import copy
def discretize(data,column_ids,n_bins=10):
    new_data = copy.deepcopy(data)
    for column in column_ids:
        k = [float(row[1][column]) for row in new_data]
        col_max = max(k);
        col_min = min(k);
        ranges = list(np.linspace(col_min, col_max, n_bins))
        for idx in range(len(data)):
            new_data[idx][1][column] = discrete_val(data[idx][1][column],ranges)
    return new_data

Sample Output

In [8]:
discretize(sample2,[2,3])

[(1, [1, 1, '30.0>', '27.0>', 1]),
 (0, [0, 0, '2.0-5.11111111111', '7.55555555556-10.3333333333', 1]),
 (1, [1, 1, '26.8888888889-30.0', '15.8888888889-18.6666666667', 2]),
 (0, [1, 1, '2.0-5.11111111111', '2.0-4.77777777778', 1]),
 (0, [0, 0, '20.6666666667-23.7777777778', '7.55555555556-10.3333333333', 0]),
 (2, [1, 0, '17.5555555556-20.6666666667', '15.8888888889-18.6666666667', 1]),
 (2, [2, 0, '14.4444444444-17.5555555556', '10.3333333333-13.1111111111', 1]),
 (2, [1, 2, '8.22222222222-11.3333333333', '24.2222222222-27.0', 1]),
 (0, [0, 1, '8.22222222222-11.3333333333', '21.4444444444-24.2222222222', 1]),
 (1, [1, 2, '5.11111111111-8.22222222222', '18.6666666667-21.4444444444', 2]),
 (0, [1, 1, '8.22222222222-11.3333333333', '7.55555555556-10.3333333333', 1]),
 (0, [0, 1, '17.5555555556-20.6666666667', '2.0-4.77777777778', 0]),
 (2, [2, 2, '5.11111111111-8.22222222222', '15.8888888889-18.6666666667', 1]),
 (2, [2, 1, '11.3333333333-14.4444444444', '4.77777777778-7.55555555556', 1

Now we write a function that will help with bootstrapping. 

In [9]:
import random
# method lifted from:http://code.activestate.com/recipes/273085-sample-with-replacement/
def sample_wr(population, k):
    "Chooses k random elements (with replacement) from a population"
    n = len(population)
    _random, _int = random.random, int  # speed hack 
    result = [None] * k
    for i in xrange(k):
        j = _int(_random() * n)
        result[i] = population[j]
    return result

And now we create the array of samples to pass to Spark.

In [10]:
# n is number of trees in our forest
n=10
length=len(sample)
forest=[sample]
# commence bootstrapping
for i in range(1,n):
    # add tree to forest
    forest.append(sample_wr(sample,length))

In [11]:
# m is the number of columns we will be computing on per tree.
m=3
def choose_columns(num_cols):
    return random.sample(range(num_cols),m)

Finally, we parallelize the forest creation.

In [12]:
#parallelize
forest_rdd=sc.parallelize(forest)
res = forest_rdd.map(lambda s: create_decision_tree(s, choose_columns(4), gain, col_sample)).collect()

Classifying using the forest generated to test accuracy

In [13]:
def get_classification(record, tree, columns):
    """
    This function recursively traverses the decision tree and returns a
    classification for the given record.
    """
    # If the current node is a string, then we've reached a leaf node and
    # we can return it as our answer
    if type(tree) == type("string"):
        return tree

    # Traverse the tree further until a leaf node is found.
    else:
        attr = tree.keys()[0]
        t = tree[attr][record[columns.index(attr)]]
        return get_classification(record, t, columns)

In [14]:
from collections import Counter
def classify_rf(data, tree_list, columns):
    res = []
    for tree in tree_list:
        try:
            res.append(get_classification(data,tree,columns))
        except:
            continue
    result = Counter(res)
    return result.most_common(1)[0][0]

Creating test data

In [15]:
test = [[1,1,0,1],
        [2,1,0,2],
        [0,0,0,0],
        [0,2,1,2],
        [1,0,0,0]]

In [16]:
test_rdd = sc.parallelize(sample).map(lambda x:x[1])
res_rdd = test_rdd.map(lambda x:(classify_rf(x, res, col_sample),x))

Collect the res_rdd to generate results

#### Testing code on bigger dataset

Get the CSV into an RDD, map it to a LabeledPoint, and then just collect it.

In [17]:
sample_data_1 = sc.textFile('../data/balance-scale.csv')
rdd_data_1=sample_data_1.map(lambda x:x.split()).map(lambda x: x[0].strip("'").split(","))\
            .map(lambda x:[v for v in x])\
            .map(lambda x: (str(x[0]),[int(k) for k in x[1:]]))
columns_data_1 = ['Left-Weight','Left-Distance','Right-Weight','Right-Distance']

In [18]:
sample3=rdd_data_1.collect()

Then repeat the process as we did before. 

In [19]:
import time
# n is number of trees in our forest
n=10
length=len(sample3)
forest3=[sample3]
# commence bootstrapping
start_time = time.time()
for i in range(1,n):
    # add tree to forest
    forest3.append(sample_wr(sample3,length))
print time.time() - start_time

0.00316286087036


In [20]:
forest3_rdd=sc.parallelize(forest3)
%time res3 = forest3_rdd.map(lambda s: create_decision_tree(s, choose_columns(4), gain, columns_data_1)).collect()

CPU times: user 5.16 ms, sys: 1.99 ms, total: 7.14 ms
Wall time: 117 ms


In [21]:
res3

[{'Right-Distance': {1: {'Left-Distance': {1: 'R',
     2: 'L',
     3: 'L',
     4: 'L',
     5: 'L'}},
   2: {'Left-Distance': {1: 'R', 2: 'R', 3: 'L', 4: 'L', 5: 'L'}},
   3: {'Left-Distance': {1: 'R', 2: 'R', 3: 'R', 4: 'L', 5: 'L'}},
   4: {'Right-Weight': {1: 'L', 2: 'R', 3: 'R', 4: 'R', 5: 'R'}},
   5: {'Right-Weight': {1: 'L', 2: 'R', 3: 'R', 4: 'R', 5: 'R'}}}},
 {'Left-Weight': {1: {'Right-Distance': {1: 'R',
     2: 'R',
     3: 'R',
     4: 'R',
     5: 'R'}},
   2: {'Right-Distance': {1: 'L', 2: 'R', 3: 'L', 4: 'R', 5: 'R'}},
   3: {'Right-Distance': {1: 'L', 2: 'L', 3: 'R', 4: 'R', 5: 'R'}},
   4: {'Left-Distance': {1: 'R', 2: 'R', 3: 'L', 4: 'L', 5: 'L'}},
   5: {'Left-Distance': {1: 'R', 2: 'L', 3: 'L', 4: 'L', 5: 'L'}}}},
 {'Right-Weight': {1: {'Left-Weight': {1: 'L',
     2: 'L',
     3: 'L',
     4: 'L',
     5: 'L'}},
   2: {'Left-Weight': {1: 'R', 2: 'L', 3: 'L', 4: 'L', 5: 'L'}},
   3: {'Right-Distance': {1: 'L', 2: 'L', 3: 'L', 4: 'R', 5: 'R'}},
   4: {'Right-Dist

Using the forest dataset

In [22]:
%time rdd_data_1.sample(True,1).collect()

CPU times: user 6.5 ms, sys: 2.57 ms, total: 9.07 ms
Wall time: 70.9 ms


[('B', [1, 1, 1, 1]),
 ('R', [1, 1, 1, 2]),
 ('R', [1, 1, 1, 4]),
 ('R', [1, 1, 1, 4]),
 ('R', [1, 1, 2, 5]),
 ('R', [1, 1, 2, 5]),
 ('R', [1, 1, 3, 1]),
 ('R', [1, 1, 3, 2]),
 ('R', [1, 1, 3, 2]),
 ('R', [1, 1, 3, 3]),
 ('R', [1, 1, 3, 4]),
 ('R', [1, 1, 3, 5]),
 ('R', [1, 1, 4, 1]),
 ('R', [1, 1, 4, 1]),
 ('R', [1, 1, 4, 3]),
 ('R', [1, 1, 4, 3]),
 ('R', [1, 1, 4, 4]),
 ('R', [1, 1, 4, 5]),
 ('R', [1, 1, 5, 1]),
 ('R', [1, 1, 5, 1]),
 ('R', [1, 1, 5, 4]),
 ('R', [1, 1, 5, 4]),
 ('R', [1, 1, 5, 5]),
 ('L', [1, 2, 1, 1]),
 ('R', [1, 2, 1, 3]),
 ('R', [1, 2, 1, 4]),
 ('B', [1, 2, 2, 1]),
 ('B', [1, 2, 2, 1]),
 ('R', [1, 2, 2, 3]),
 ('R', [1, 2, 2, 3]),
 ('R', [1, 2, 2, 5]),
 ('R', [1, 2, 3, 2]),
 ('R', [1, 2, 3, 3]),
 ('R', [1, 2, 3, 5]),
 ('R', [1, 2, 3, 5]),
 ('R', [1, 2, 4, 1]),
 ('R', [1, 2, 4, 2]),
 ('R', [1, 2, 4, 3]),
 ('R', [1, 2, 4, 3]),
 ('R', [1, 2, 4, 3]),
 ('R', [1, 2, 4, 3]),
 ('R', [1, 2, 4, 4]),
 ('R', [1, 2, 4, 4]),
 ('R', [1, 2, 4, 4]),
 ('R', [1, 2, 4, 5]),
 ('R', [1,

In [23]:
txtFile=sc.textFile('../data/covtype.csv')
#Convert it into RDD of lists 
rdd=(txtFile.map(lambda x:x.split())
    .map(lambda x: x[0].strip("'").split(","))
    .map(lambda x:[float(v) for v in x])
    .map(lambda x: (x[-1]-1,x[0:-1])))

In [24]:
import csv

In [25]:
data = []
with open('../data/covtype.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        data.append((row[-1],row[0:-1]))

In [26]:
import itertools
soil_list =[]
for k in range(40):
    string = 'Soil_Type_' + str(k+1)
    soil_list.append(string)
WA_list =[]
for k in range(4):
    string = 'WA_' + str(k+1)
    WA_list.append(string)
names = [['Elevation'], ['Aspect'], ['Slope'], ['HDHyrdo'], ['VDHydro'], ['HDRoadways'], \
         ['9amHills'],['NoonHills'], ['3pmHills'], ['HDFirePoints'], WA_list,\
         soil_list, ['Cover_Type']]
columns_data = list(itertools.chain(*names))

In [28]:
data_train = discretize(data,range(10))

In [29]:
import time
# n is number of trees in our forest
n=3
length=len(data_train)
forest3=[data_train]
# commence bootstrapping
start_time = time.time()
for i in range(1,n):
    # add tree to forest
    forest3.append(sample_wr(data_train,length))
print time.time() - start_time

0.550888061523


In [ ]:
forest3_rdd=sc.parallelize(forest3)


In [ ]:
%time res3 = forest3_rdd.map(lambda s: create_decision_tree(s, choose_columns(2), gain, columns_data)).collect()